In [1]:
#import dependencies 
import pandas as pd
import requests
import json


In [2]:
#input data 
DATA_PATH = "data/"

tasks_df = pd.read_csv(DATA_PATH+"tasks_data.csv")
fleet_df = pd.read_csv(DATA_PATH+"fleet_data.csv")


In [3]:
tasks_df

,Task_ID,Latitude,Longitude,Demand
0,Base_1,29.737497,-95.564608,0
1,Base_2,29.737137,-95.480530,0
2,Task_1,29.718221,-95.419154,50
3,Task_2,29.747544,-95.373544,30
4,Task_3,29.762797,-95.360739,40
5,Task_4,29.732337,-95.441343,20
6,Task_5,29.742487,-95.366535,30
7,Task_6,29.750734,-95.369664,25
8,Task_7,29.762995,-95.382217,30
9,Task_8,29.751114,-95.357903,40


In [4]:
fleet_df

,Truck_ID,Truck_Type,Capacity,Fuel_Cost_Per_Mile,Base_Location
0,Diesel_1,Diesel,500,0.15,Base_1
1,Diesel_2,Diesel,500,0.15,Base_2
2,Gas_1,Gas,400,0.12,Base_1
3,Gas_2,Gas,400,0.12,Base_1
4,Gas_3,Gas,400,0.12,Base_1
5,Gas_4,Gas,400,0.12,Base_2
6,Gas_5,Gas,400,0.12,Base_2
7,Gas_6,Gas,400,0.12,Base_2
8,Gas_7,Gas,400,0.12,Base_1
9,Gas_8,Gas,400,0.12,Base_2


In [ ]:
import requests
import pandas as pd
import json

def build_cost_matrix(df, diesel_cost_per_mile=0.15, gas_cost_per_mile=0.12):
    """
    Generate cost matrices for diesel and gas trucks using the OSRM API.

    Parameters:
    - df (DataFrame): DataFrame containing columns ['Task_ID', 'Latitude', 'Longitude'].
    - diesel_cost_per_mile (float): Fuel cost per mile for diesel trucks.
    - gas_cost_per_mile (float): Fuel cost per mile for gas trucks.

    Returns:
    - diesel_cost_matrix (DataFrame): Cost matrix for diesel trucks.
    - gas_cost_matrix (DataFrame): Cost matrix for gas trucks.
    """
    try:
        # Extract latitude and longitude from DataFrame
        latitude = df["Latitude"].to_numpy()
        longitude = df["Longitude"].to_numpy()
        
        # Prepare locations for the OSRM API
        locations = ""
        n_orders = len(df)
        for i in range(n_orders):
            locations = locations + "{},{};".format(longitude[i], latitude[i])
        
        # Make the OSRM API request
        response = requests.get("http://router.project-osrm.org/table/v1/driving/" + locations[:-1])
        
        if response.status_code == 200:
            routes = json.loads(response.content)
            
            # Extract the distance matrix (in meters)
            distance_matrix = routes['distances']  # Returns distances in meters
            
            # Create a DataFrame for better visualization
            coords_index = {i: (latitude[i], longitude[i]) for i in range(df.shape[0])}
            distance_matrix_df = pd.DataFrame(distance_matrix, index=df["Task_ID"], columns=df["Task_ID"])
            
            # Calculate cost matrices (convert meters to miles)
            diesel_cost_matrix = distance_matrix_df * diesel_cost_per_mile / 1609.34
            gas_cost_matrix = distance_matrix_df * gas_cost_per_mile / 1609.34
            
            return diesel_cost_matrix, gas_cost_matrix
        else:
            print(f"Error: Unable to fetch data from OSRM API. Status code: {response.status_code}")
            return None, None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None, None


In [6]:
diesel_matrix, gas_matrix = build_cost_matrix(tasks_df)


In [7]:
diesel_matrix

Task_ID,Base_1,Base_2,Task_1,Task_2,Task_3,Task_4,Task_5,Task_6,Task_7,Task_8,Task_9,Task_10
Task_ID,,,,,,,,,,,,
Base_1,0.000000,0.776955,1.662756,2.020173,2.252927,1.356043,2.141639,2.077327,2.277440,2.804193,2.144379,1.880644
Base_2,0.797265,0.000000,0.834836,1.192262,1.425016,0.442514,1.313719,1.249407,1.449520,1.976282,1.316459,0.796174
Task_1,1.666978,0.895252,0.000000,0.616017,0.848770,0.377027,0.737473,0.637062,0.873274,1.400037,0.740223,0.479442
Task_2,2.024936,1.253209,0.602707,0.000000,0.213814,0.735283,0.117011,0.057145,0.288687,0.661352,0.155626,0.314365
Task_3,2.224257,1.452540,0.836430,0.223256,0.000000,0.934604,0.316490,0.219164,0.214476,0.415670,0.202052,0.513686
Task_4,1.358951,0.450054,0.385248,0.760029,0.992783,0.000000,0.881486,0.817183,1.017287,1.544049,0.884235,0.528338
Task_5,2.099873,1.328156,0.712047,0.122976,0.302668,0.810220,0.000000,0.125772,0.377531,0.793863,0.244479,0.390551
Task_6,2.095371,1.323645,0.658695,0.121634,0.213926,0.805718,0.134701,0.000000,0.265292,0.543763,0.126182,0.389200
Task_7,2.168398,1.088002,0.797526,0.563439,0.624899,0.803687,0.656682,0.604282,0.000000,1.002094,0.469021,0.396591


In [8]:
gas_matrix

Task_ID,Base_1,Base_2,Task_1,Task_2,Task_3,Task_4,Task_5,Task_6,Task_7,Task_8,Task_9,Task_10
Task_ID,,,,,,,,,,,,
Base_1,0.000000,0.621564,1.330205,1.616138,1.802341,1.084835,1.713311,1.661861,1.821952,2.243354,1.715503,1.504515
Base_2,0.637812,0.000000,0.667869,0.953810,1.140013,0.354011,1.050975,0.999525,1.159616,1.581026,1.053167,0.636939
Task_1,1.333583,0.716202,0.000000,0.492813,0.679016,0.301622,0.589979,0.509650,0.698619,1.120029,0.592178,0.383554
Task_2,1.619949,1.002568,0.482165,0.000000,0.171051,0.588226,0.093609,0.045716,0.230949,0.529081,0.124501,0.251492
Task_3,1.779405,1.162032,0.669144,0.178605,0.000000,0.747683,0.253192,0.175332,0.171581,0.332536,0.161641,0.410949
Task_4,1.087161,0.360043,0.308198,0.608023,0.794226,0.000000,0.705188,0.653746,0.813829,1.235239,0.707388,0.422670
Task_5,1.679899,1.062525,0.569637,0.098381,0.242134,0.648176,0.000000,0.100618,0.302024,0.635090,0.195583,0.312441
Task_6,1.676297,1.058916,0.526956,0.097307,0.171141,0.644575,0.107761,0.000000,0.212234,0.435011,0.100946,0.311360
Task_7,1.734719,0.870402,0.638021,0.450751,0.499919,0.642949,0.525346,0.483426,0.000000,0.801675,0.375217,0.317273


In [ ]:
#set task data
